# **정수 인코딩(Integer Encoding)**
컴퓨터는 텍스트보다는 숫자를 더 잘 처리 할 수 있습니다. 이를 위해 자연어 처리에서는 텍스트를 숫자로 바꾸는 여러가지 기법들이 있습니다. 그리고 그러한 기법들을 본격적으로 적용시키기 위한 첫 단계로 각 단어를 고유한 숫자에 맵핑(mapping)시키는 전처리 작업이 필요할 때가 있습니다.

<br>

예를 들어 갖고 있는 텍스트에 단어가 5,000개가 있다면, 5,000개의 단어들 각각에 0번부터 4,999번까지 단어와 맵핑되는 고유한 숫자, 다른 말로는 인덱스를 부여합니다. (1번부터 시작해서 부여해도 상관은 없으며 코딩을 하는 동안 이를 인지하고 있으면 됩니다.) 가령, book은 150번, dog는 171번, love는 192번, books는 212번과 같이 숫자가 부여됩니다. 인덱스를 부여하는 방법은 여러 가지가 있을 수 있는데 랜덤으로 부여하기도 하지만, 보통은 전처리도 같이 겸하기 위해 단어에 대한 빈도수로 정렬한 뒤에 부여합니다.

## **1. 정수 인코딩(Integer Encoding)**
왜 이러한 작업이 필요한 지에 대해서는 뒤에서 원-핫 인코딩과 이를 이용한 여러가지 딥 러닝 실습이나 워드 임베딩 챕터에서 배워보기로 하고, 여기서는 단어에 어떻게 순차적으로 인덱스를 부여하는지에 대해서만 정리하겠습니다.

<br>

단어에 숫자를 부여하는 방법 중 하나로 단어를 빈도수 순으로 정렬하여 단어 집합(vocabulary)을 만들고, 빈도수가 높은 순서대로 차례로 낮은 숫자부터 정수를 부여하는 방법이 있습니다. 이해를 돕기위해 단어의 빈도수가 적당하게 분포되도록 의도적으로 만든 작은 텍스트 데이터를 가지고 빈도수 순서대로 단어를 정수로 바꿔보도록 하겠습니다.

In [1]:
text="A barber is a person. a barber is good person. a barber is huge person. he Knew A Secret! The Secret He Kept is huge secret. Huge secret. His barber kept his word. a barber kept his word. His barber kept his secret. But keeping and keeping such a huge secret to himself was driving the barber crazy. the barber went up a huge mountain."

우선 여러 문장이 함께 있는 텍스트 데이터로부터 문장 토큰화를 수행해보도록 하겠습니다.

In [2]:
# 문장 토큰화
from nltk.tokenize import sent_tokenize

text = sent_tokenize(text)
print(text)

['A barber is a person.', 'a barber is good person.', 'a barber is huge person.', 'he Knew A Secret!', 'The Secret He Kept is huge secret.', 'Huge secret.', 'His barber kept his word.', 'a barber kept his word.', 'His barber kept his secret.', 'But keeping and keeping such a huge secret to himself was driving the barber crazy.', 'the barber went up a huge mountain.']


기존의 텍스트 데이터가 문장 단위로 토큰화 된 것을 확인할 수 있습니다. 이제 정제 작업을 병행하며, 단어 토큰화를 수행해보도록 하겠습니다. 또한, 단어 토큰화를 수행하면서 각 단어에 대한 빈도수 또한 같이 계산합니다.

In [3]:
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from collections import Counter

# 파이썬의 Counter 모듈을 이용하면 단어의 모든 빈도를 쉽게 계산할 수 있습니다.
vocab = Counter()

sentences = []
stop_words = set(stopwords.words('english'))

# text는 문장 토큰화 리스트
for i in text:
    sentence = word_tokenize(i) # 단어 토큰화를 수행합니다.
    result = []
    for word in sentence:
        word = word.lower() # 모든 단어를 소문자화하여 단어의 개수를 줄입니다.
        if word not in stop_words: # 단어 토큰화 된 결과에 대해서 불용어를 제거합니다.
            if len(word) > 2: # 단어 길이가 2이하인 경우에 대하여 추가로 단어를 제거합니다.
                result.append(word)
                vocab[word] = vocab[word] + 1 #각 단어의 빈도를 Count 합니다.
                
    sentences.append(result)
print(sentences)

[['barber', 'person'], ['barber', 'good', 'person'], ['barber', 'huge', 'person'], ['knew', 'secret'], ['secret', 'kept', 'huge', 'secret'], ['huge', 'secret'], ['barber', 'kept', 'word'], ['barber', 'kept', 'word'], ['barber', 'kept', 'secret'], ['keeping', 'keeping', 'huge', 'secret', 'driving', 'barber', 'crazy'], ['barber', 'went', 'huge', 'mountain']]


텍스트를 숫자로 바꾸는 단계라는 것은 이제부터 본격적으로 여러가지 알고리즘을 사용하여 자연어 처리 작업에 들어간다는 의미이므로, ***단어가 텍스트일 때만 할 수 있는 최대한의 전처리***를 끝내놓아야 합니다.

우선, 동일한 단어가 대문자로 표기되었다는 이유로 서로 다른 단어로 카운트되는 일이 없도록 모든 단어를 소문자로 바꾸었습니다. 그리고 자연어 처리에서 크게 의미를 갖지 못하는 단어 또한 카운트에서 제외시키고자 불용어 제거와 길이가 짧은 단어를 제거하는 방법을 사용하였습니다.

In [4]:
print(vocab)

Counter({'barber': 8, 'secret': 6, 'huge': 5, 'kept': 4, 'person': 3, 'word': 2, 'keeping': 2, 'good': 1, 'knew': 1, 'driving': 1, 'crazy': 1, 'went': 1, 'mountain': 1})


이제 vocab에는 각 단어에 대한 빈도수가 기록되었습니다. 각 단어들을 빈도수 순으로 정렬해보도록 하겠습니다.

In [5]:
vocab_sorted = sorted(vocab.items(), key = lambda x:x[1], reverse = True)
print(vocab_sorted)

[('barber', 8), ('secret', 6), ('huge', 5), ('kept', 4), ('person', 3), ('word', 2), ('keeping', 2), ('good', 1), ('knew', 1), ('driving', 1), ('crazy', 1), ('went', 1), ('mountain', 1)]


이제 각 단어가 등장 빈도수 순으로 정렬된 것을 확인할 수 있습니다.

In [6]:
word_to_index = {}
i = 0

for (word, frequency) in vocab_sorted:
    if frequency > 1: # 정제(Cleaning) 챕터에서 언급했듯이 빈도수가 적은 단어는 제외한다.
        i = i + 1
        word_to_index[word] = i
print(word_to_index)

{'barber': 1, 'secret': 2, 'huge': 3, 'kept': 4, 'person': 5, 'word': 6, 'keeping': 7}


위 코드처럼 이제 높은 빈도수에 따라서 인덱스를 부여합니다. 즉, 1의 인덱스를 가진 단어가 가장 빈도수가 높은 단어가 됩니다. 그리고 이러한 작업을 수행하는 동시에 각 단어의 빈도수를 알 경우에만 할 수 있는 전처리인 빈도수가 적은 단어를 제외시키는 작업을 합니다. 바로 단어의 빈도수를 계산할 때의 이점이 여기에 있습니다. 등장 빈도가 낮은 단어는 자연어 처리에서 의미를 가지지 않을 가능성이 높기 때문입니다. 여기서는 빈도수가 1인 단어들은 전부 제외시켰습니다.

## **2. 케라스(Keras)의 텍스트 전처리**
케라스(Keras)는 기본적인 전처리를 위한 도구들을 제공합니다. 때로는 정수 인코딩을 위해서 케라스의 전처리 도구인 토크나이저를 사용하기도 하는데, 사용 방법과 그 한계점에 대해서 이해보도록 하겠습니다.

In [7]:
from keras.preprocessing.text import Tokenizer

text = ["A barber is a person. a barber is good person. a barber is huge person. he Knew A Secret! The Secret He Kept is huge secret. Huge secret. His barber kept his word. a barber kept his word. His barber kept his secret. But keeping and keeping such a huge secret to himself was driving the barber crazy. the barber went up a huge mountain."]
t = Tokenizer()
t.fit_on_texts(text)

Using TensorFlow backend.


fit_on_texts는 텍스트의 리스트(list)를 가지고 단어 빈도수에 기반한 사전을 만듭니다. 빈도수 순으로 단어에게 인덱스를 부여하는데, 정확하게 위에서 설명한 정수 인코딩 작업이 이루어진다고 보면됩니다. 각 단어에 인덱스가 어떻게 부여되었는지를 보려면, word_index를 사용합니다.

In [8]:
print(t.word_index)

{'a': 1, 'barber': 2, 'secret': 3, 'huge': 4, 'his': 5, 'is': 6, 'kept': 7, 'person': 8, 'the': 9, 'he': 10, 'word': 11, 'keeping': 12, 'good': 13, 'knew': 14, 'but': 15, 'and': 16, 'such': 17, 'to': 18, 'himself': 19, 'was': 20, 'driving': 21, 'crazy': 22, 'went': 23, 'up': 24, 'mountain': 25}


각 단어의 빈도수가 높은 순서대로 인덱스가 부여되기 때문에, a가 1번 인덱스를 가지는 것을 볼 수 있습니다. 각 단어가 몇 개였는지 카운트 된 결과를 보려면 word_counts를 사용합니다.

In [9]:
print(t.word_counts)

OrderedDict([('a', 8), ('barber', 8), ('is', 4), ('person', 3), ('good', 1), ('huge', 5), ('he', 2), ('knew', 1), ('secret', 6), ('the', 3), ('kept', 4), ('his', 5), ('word', 2), ('but', 1), ('keeping', 2), ('and', 1), ('such', 1), ('to', 1), ('himself', 1), ('was', 1), ('driving', 1), ('crazy', 1), ('went', 1), ('up', 1), ('mountain', 1)])


위를 보면 각 단어가 몇 개였는지를 카운트한 결과를 보여줄 뿐만 아니라, 가장 등장 빈도 수가 높은 순서대로 출력합니다. 이 경우에는 a와 barber가 둘 다 8회 등장하였으나 알파벳 순서로 인해 a가 먼저 등장하였습니다.

아래는 texts_to_sequences()는 입력으로 들어온 코퍼스에 대해서 각 단어를 word_index에서 이미 정해진 인덱스로 변환하여 변환하여 출력합니다.

In [10]:
print(t.texts_to_sequences(text))

[[1, 2, 6, 1, 8, 1, 2, 6, 13, 8, 1, 2, 6, 4, 8, 10, 14, 1, 3, 9, 3, 10, 7, 6, 4, 3, 4, 3, 5, 2, 7, 5, 11, 1, 2, 7, 5, 11, 5, 2, 7, 5, 3, 15, 12, 16, 12, 17, 1, 4, 3, 18, 19, 20, 21, 9, 2, 22, 9, 2, 23, 24, 1, 4, 25]]


Counter()를 사용했을 때의 최종 결과와는 달리 총 25개의 단어가 모두 존재하고 25개의 인덱스가 모두 나옵니다. 별도로 빈도수에 기반한 단어들을 삭제하거나, 불용어 처리를 해주지 않았기 때문입니다. 사실 위의 코드에서 t = Tokenizer() 대신 t = Tokenizer(num_words=숫자)과 같은 방법으로 빈도수가 높은 상위 몇 개의 단어만 남기고 진행시키는 방법이 케라스 토크나이저에도 공식적으로 존재는 하지만, 이를 사용하면 t.texts_to_sequences(text)에서는 적용이 되면서 t_word_index와 t.word_counts에는 여전히 모든 단어가 인식되는 등의 사용자에게 혼란을 주는 문제가 존재합니다. 물론, 정확하게 이를 이해하고 사용한다면 큰 문제는 없습니다.

케라스 토크나이저의 도구를 사용하면서 사용자가 추가로 정제를 할 수는 있지만, 케라스 전처리 도구에 맞추어서 전처리를 해주는 작업이 직관적이지 않기 때문에 Counter() 등의 다른 방법을 사용할 때보다 오히려 더 복잡해질 수도 있습니다. 예를 들어 빈도수가 1인 단어에 대해서 word_index와 word_counts까지 고려하여 아예 제거하는 전처리 방법은 다음과 같이 수행할 수 있습니다.

In [11]:
# 빈도수가 2미만 단어를 w라고 저장
words_frequency = [w for w, c in t.word_counts.items() if c < 2]

for w in words_frequency:
    del t.word_index[w] # 해당 단어에 대한 인덱스 정보를 삭제
    del t.word_counts[w] # 해당 단어에 대한 카운트 정보를 삭제

print(t.texts_to_sequences(text))
print(t.word_index)

[[1, 2, 6, 1, 8, 1, 2, 6, 8, 1, 2, 6, 4, 8, 10, 1, 3, 9, 3, 10, 7, 6, 4, 3, 4, 3, 5, 2, 7, 5, 11, 1, 2, 7, 5, 11, 5, 2, 7, 5, 3, 12, 12, 1, 4, 3, 9, 2, 9, 2, 1, 4]]
{'a': 1, 'barber': 2, 'secret': 3, 'huge': 4, 'his': 5, 'is': 6, 'kept': 7, 'person': 8, 'the': 9, 'he': 10, 'word': 11, 'keeping': 12}


## **3. enumerate**
파이썬의 enumerate()를 사용하면 단어의 리스트에 대해서 쉽게 인덱스를 부여할 수 있습니다. enumerate()는 주어진 입력에 인덱스를 부여하여 인덱스도 함께 리턴해준다는 특징이 있습니다. 우선 예제를 통해 enumerate()를 이해해보도록 하겠습니다.

In [12]:
test=[8, 2, 5, 1, 3, 7, 9, 4, 6, 10]

for index, value in enumerate(test):
    print("index : {}, value : {}".format(index, value))

index : 0, value : 8
index : 1, value : 2
index : 2, value : 5
index : 3, value : 1
index : 4, value : 3
index : 5, value : 7
index : 6, value : 9
index : 7, value : 4
index : 8, value : 6
index : 9, value : 10


즉, 단어를 우선 정렬 하고나서 enumerate()를 사용하면 정수 인코딩 결과 자체는 굉장히 쉽게 얻을 수 있습니다. 주어진 데이터로부터 단어를 빈도수로 정렬하고, enumerate()로 정수 인코딩을 하는 과정까지 진행해보도록 하겠습니다.

In [13]:
# 문장 토큰화까지는 되어 있다고 가정함
text=[['barber', 'person'], ['barber', 'good', 'person'], ['barber', 'huge', 'person'], ['knew', 'secret'], ['secret', 'kept', 'huge', 'secret'], ['huge', 'secret'], ['barber', 'kept', 'word'], ['barber', 'kept', 'word'], ['barber', 'kept', 'secret'], ['keeping', 'keeping', 'huge', 'secret', 'driving', 'barber', 'crazy'], ['barber', 'went', 'huge', 'mountain']]

단어 집합(vocabulary)을 만들기 위해서 문장의 경계인 [, ]를 제거하고 단어들을 하나의 리스트로 만들겠습니다.

In [14]:
words = sum(text, [])
print(words)

['barber', 'person', 'barber', 'good', 'person', 'barber', 'huge', 'person', 'knew', 'secret', 'secret', 'kept', 'huge', 'secret', 'huge', 'secret', 'barber', 'kept', 'word', 'barber', 'kept', 'word', 'barber', 'kept', 'secret', 'keeping', 'keeping', 'huge', 'secret', 'driving', 'barber', 'crazy', 'barber', 'went', 'huge', 'mountain']


이제 단어를 빈도순으로 정렬하고, 중복도 제거한 결과를 만들어보겠습니다.

In [15]:
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from collections import Counter

# 파이썬의 Counter 모듈을 이용하면 단어의 모든 빈도를 쉽게 계산할 수 있습니다.
vocab = Counter()

# stop words load
stop_words = set(stopwords.words('english'))

for word in words:
    word = word.lower()
    if word not in stop_words:
        vocab[word] = vocab[word] + 1
        
print(vocab)

Counter({'barber': 8, 'secret': 6, 'huge': 5, 'kept': 4, 'person': 3, 'word': 2, 'keeping': 2, 'good': 1, 'knew': 1, 'driving': 1, 'crazy': 1, 'went': 1, 'mountain': 1})


enumerate()를 사용하기 위한 전처리가 끝났습니다. 이제 enumerate()를 통해 순서대로 인덱스를 부여하기만 하면 됩니다.

In [16]:
# 인덱스를 0이 아닌 1부터 부여.
word_to_index = {word : index + 1 for index, word in enumerate(vocab)}
print(word_to_index)

{'barber': 1, 'person': 2, 'good': 3, 'huge': 4, 'knew': 5, 'secret': 6, 'kept': 7, 'word': 8, 'keeping': 9, 'driving': 10, 'crazy': 11, 'went': 12, 'mountain': 13}


## **4. NLTK의 FreqDist 클래스**
NLTK에서는 토큰들의 빈도를 손쉽게 셀 수 있도록 빈도수 계산 클래스인 FreqDist를 지원합니다. FreqDist 클래스는 토큰들을 입력으로 받아 해당 토큰의 빈도 정보를 계산합니다. **즉, FreqDist의 입력으로는 반드시 토큰화가 이루어진 상태여야 합니다.**

In [17]:
test_list = ['barber', 'barber', 'person', 'barber', 'good', 'person']

from nltk import FreqDist
fdist = FreqDist(test_list)

이제 fdist에는 각 단어들의 빈도 정보가 저장되었습니다. 이를 다양한 형태의 정보로 출력해보도록 하겠습니다. ***FreqDist 클래스는 단어를 키(key), 출현빈도를 값(value)으로 가지는 파이썬의 딕셔너리(dict) 자료형의 형태***를 가집니다. 즉, 단어를 입력하면 출현빈도의 값을 얻을 수 있습니다.

In [18]:
# 전체 단어의 개수인 6 출력
fdist.N()

6

In [19]:
# 'barber'라는 단어의 확률.
# 총 6개의 단어 중 3번 등장하였으므로 0.5의 값 출력
fdist.freq("barber")

0.5

In [20]:
# 'barber'라는 단어의 빈도수 출력
fdist["barber"]

3

In [21]:
# 등장 빈도수가 가장 높은 barber와 person이 각각의 빈도수와 함께 출력
fdist.most_common(2)

[('barber', 3), ('person', 2)]

---
# **원-핫 인코딩(One-hot encoding)**
컴퓨터는 문자보다는 숫자를 더 잘 처리 할 수 있습니다. 이를 위해 자연어 처리에서는 문자를 숫자로 바꾸는 여러가지 기법들이 있습니다. 원-핫 인코딩(One-hot encoding)은 그 많은 기법 중에서 단어를 표현하는 가장 기본적인 표현 방법이며, 머신 러닝, 딥 러닝을 하기 위해서는 반드시 배워야 하는 표현 방법입니다.
<br>

원-핫 인코딩에 대해서 배우기에 앞서 **단어 집합(vocabulary)**에 대해서 정의해보도록 하겠습니다. 단어 집합은 앞으로 자연어 처리에서 계속 나오는 개념이기 때문에 여기서 이해하고 가야합니다. 단어 집합은 서로 다른 단어들의 집합입니다. 여기서 혼동이 없도록 서로 다른 단어라는 정의에 대해서 좀 더 주목할 필요가 있습니다.
<br>

**단어 집합(vocabulary)에서는 기본적으로 book과 books와 같이 단어의 변형 형태도 다른 단어로 간주합니다.** 이 책에서는 앞으로 단어 집합에 있는 단어들을 가지고, 문자를 숫자(더 구체적으로는 벡터)로 바꾸는 원-핫 인코딩을 포함한 여러 방법에 대해서 배우게 됩니다.
<br>

원-핫 인코딩을 위해서 먼저 해야할 일은 갖고 있는 우선 단어 집합을 만드는 일입니다. 텍스트의 모든 단어를 중복을 허락하지 않고 모아놓는다면 이를 단어 집합이라고 합니다. 그리고 이 단어 집합에 고유한 숫자를 부여하는 일을 진행합니다. 갖고 있는 텍스트에 단어가 5,000개가 등장했다면, 단어 집합의 크기를 5,000이라고 합니다. 5,000개의 단어가 있는 이 단어 집합의 단어들마다 0번부터 4,999번까지 인덱스를 부여한다고 해보겠습니다. 가령, book은 150번, dog는 171번, love는 192번, books는 212번과 같이 부여됩니다. (정수 인코딩 챕터 참고)
<br>

이제 각 단어에 고유한 인덱스를 붙였다고 합시다. 이 숫자로 바뀐 단어들을 벡터로 다루고 싶다면 어떻게 하면 될까요?

## **1. 원-핫 인코딩(One-hot encoding)이란?**

원-핫 인코딩은 단어 집합의 크기를 벡터의 차원으로 하고, 표현하고 싶은 단어의 인덱스에 1의 값을 부여하고, 다른 인덱스에는 0을 부여하는 단어의 벡터 표현 방식입니다. 이렇게 표현된 벡터를 원-핫 벡터(One-hot vector)라고 합니다.
<br>

원-핫 인코딩의 과정을 두 가지 과정으로 정리해보겠습니다.

1. 각 단어에 고유한 인덱스를 부여합니다. (정수 인코딩)
2. 표현하고 싶은 단어의 인덱스의 위치에 1을 부여하고, 다른 단어의 인덱스의 위치에는 0을 부여합니다.
<br>

이해를 돕기 위해서 한국어 문장을 예제로, 실제로 원-핫 벡터를 만들어보도록 하겠습니다.

* **문장 : 나는 자연어 처리를 배운다**

위 문장에 대해서 원-핫 인코딩을 진행하는 코드는 아래와 같습니다.

In [22]:
from konlpy.tag import Okt
okt = Okt()

token = okt.morphs("나는 자연어 처리를 배운다")
print(token)

['나', '는', '자연어', '처리', '를', '배운다']


위 코드처럼 코엔엘파이의 Okt 형태소 분석기를 통해서 우선 문장에 대해서 형태소 토큰화를 수행하였습니다.

In [23]:
word2index = {}

for voca in token:
    if voca not in word2index.keys():
        word2index[voca] = len(word2index)
print(word2index)

{'나': 0, '는': 1, '자연어': 2, '처리': 3, '를': 4, '배운다': 5}


위처럼 각 토큰에 대해서 고유한 인덱스(index)를 부여하였습니다. 지금은 문장이 짧기 때문에 각 단어의 빈도수를 고려하지 않지만, 빈도수 순대로 단어를 정렬하여 고유한 인덱스를 부여하는 작업이 사용되기도 합니다. *(정수 인코딩 챕터 참고)*

In [24]:
[0] * len(word2index)

[0, 0, 0, 0, 0, 0]

In [25]:
def one_hot_encoding(word, word2index):
    # [0] * len(word2index) -> [0, 0, 0, 0, 0, 0]
    one_hot_vector = [0] * (len(word2index))
    index = word2index[word]
    one_hot_vector[index] = 1
    return one_hot_vector

위에서 토큰을 입력하면 해당 토큰에 대한 원-핫 벡터를 만들어내는 함수를 만들었습니다.

In [26]:
one_hot_encoding('자연어', word2index)

[0, 0, 1, 0, 0, 0]

해당 함수에 '자연어'라는 토큰을 입력으로 넣어봤더니 [0, 0, 1, 0, 0, 0]라는 벡터가 나왔습니다. 자연어는 단어 집합에서 인덱스가 2이므로, 자연어를 표현하는 원-핫 벡터는 인덱스 2의 값이 1이며, 나머지 값은 0인 벡터가 나옵니다.

## **2. 케라스(Keras)를 이용한 원-핫 인코딩(One-hot encoding)**
위에서는 원-핫 인코딩을 이해하기 위해 파이썬으로 직접 코드를 작성하였지만, ***케라스는 자동으로 원-핫 인코딩을 만들어 주는 유용한 도구로서 to_categorical()를 지원***합니다. 이번에는 케라스만으로 정수 인코딩과 원-핫 인코딩을 순차적으로 진행해보도록 하겠습니다.

In [27]:
text="나랑 점심 먹으러 갈래 점심 메뉴는 햄버거 갈래 갈래 햄버거 최고야"

위와 같은 문장이 있다고 했을 때, 정수 인코딩 챕터에서 배운 케라스의 토크나이저 도구를 이용한 정수 인코딩(인덱스 부여)을 진행하면 다음과 같습니다.

In [28]:
from keras_preprocessing.text import Tokenizer

t = Tokenizer()

# 입력으로 [text]가 아닌 text를 넣을 경우 한 글자 단위 인코딩이 되버립니다. ex 갈 : 1, 래 : 2
t.fit_on_texts([text])

# 각 단어에 대한 인코딩 결과 출력.
print(t.word_index)

{'갈래': 1, '점심': 2, '햄버거': 3, '나랑': 4, '먹으러': 5, '메뉴는': 6, '최고야': 7}


위와 같이 생성된 단어 집합(Vocabulary)에 있는 단어들로만 구성된 텍스트가 있다면, texts_to_sequences()를 통해서 이를 바로 인덱스의 나열로 변환가능합니다. 생성된 단어 집합의 단어들로만 구성된 서브 텍스트 text2를 가지고 확인해보겠습니다.

In [29]:
text2="점심 먹으러 갈래 메뉴는 햄버거 최고야"
x = t.texts_to_sequences([text2])
x = sum(x, [])
print(x)

[2, 5, 1, 6, 3, 7]


지금까지 진행한 것은 이미 정수 인코딩 챕터에서 배운 정수 인코딩 과정입니다. 이제 해당 결과를 가지고, 원-핫 인코딩을 진행해보도록 하겠습니다. 케라스는 정수 인코딩 된 결과를 입력으로 받아 바로 원-핫 인코딩 과정을 수행하는 to_categorical()를 지원합니다.

In [30]:
# 단어 집합의 크기. 이 경우는 단어의 개수가 7이므로 7.
vocab_size = len(t.word_index)

from keras.utils import to_categorical

# 실제 단어 집합의 크기보다 +1로 크기를 만들어야함.
x = to_categorical(x, num_classes = vocab_size + 1)

print(x)

[[0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1.]]


여기서 **주의할 점**은 to_categorical()은 정수 인코딩으로 부여된 인덱스를 그대로 배열의 인덱스로 사용하기 때문에, ***t.fit_on_texts()를 사용하여 정수 인코딩을 하였을 경우에는 실제 단어 집합의 크기보다 +1의 크기를 인자로 주어야 한다는 점***입니다.

***t.fit_on_texts()는 인덱스를 1부터 부여***합니다. 하지만 배열의 인덱스는 0부터 시작하므로 맨 마지막 인덱스를 가진 단어의 인덱스가 7인데, 이를 원-핫 벡터로 만들기 위해서는 총 8의 크기를 가진 배열이 필요합니다.

위의 결과는 "점심 먹으러 갈래 메뉴는 햄버거 최고야"라는 문장이 [2, 5, 1, 6, 3, 7]로 정수 인코딩이 되고나서, 각각의 인코딩 된 결과를 인덱스로 원-핫 인코딩이 수행된 모습을 보여줍니다.

## **3. 원-핫 인코딩(One-hot encoding)의 한계**

이러한 표현 방식은 ***단어의 개수가 늘어날 수록, 벡터를 저장하기 위해 필요한 공간이 계속 늘어난다는 단점***이 있습니다. 다른 말로는 벡터의 차원이 계속 늘어난다고도 표현합니다. 원 핫 벡터는 단어 집합의 크기가 곧 벡터의 차원 수가 됩니다. ***가령, 단어가 1,000개인 코퍼스를 가지고 원 핫 벡터를 만들면, 모든 단어 각각은 모두 1,000개의 차원을 가진 벡터가 됩니다. 다시 말해 모든 단어 각각은 하나의 값만 1을 가지고, 999개의 값은 0의 값을 가지는 벡터가 되는데, 이는 매우 비효율적인 표현 방법***입니다.

또한 이런 방식은 ***단어의 유사성을 전혀 표현하지 못한다는 단점***이 있습니다. 예를 들어서 늑대, 호랑이, 강아지, 고양이라는 4개의 단어에 대해서 원-핫 인코딩을 해서 각각, [1, 0, 0, 0], [0, 1, 0, 0], [0, 0, 1, 0], [0, 0, 0, 1]이라는 원-핫 벡터를 부여받았다고 합시다. 이 때 이 원-핫 벡터 표현 방법을 통해서는 강아지와 늑대가 유사하고, 호랑이와 고양이가 유사하다는 것을 표현할 수가 없습니다. 좀 더 극단적인 예를 들어보겠습니다. 원-핫 벡터로는 유사성을 표현할 수 없기 때문에, 강아지와 개라는 단어 유사 정도나 강아지와 냉장고라는 뜬금없는 유사 관계나 차이를 알아볼 수가 없습니다.

***단어 간 유사성을 표현, 계산할 수 없다는 단점은 검색 시스템 등에서 심각한 문제***입니다. 가령, 여행을 가려고 웹 검색창에 '삿포로 숙소'라는 단어를 검색한다고 합시다. 제대로 된 검색 시스템이라면, '삿포로 숙소'라는 검색어에 대해서 '삿포로 게스트 하우스', '삿포로 료칸', '삿포로 호텔'과 같은 유사 단어에 대한 결과도 함께 보여줄 수 있어야 합니다. 하지만 단어간 유사성을 계산할 수 없다면, '게스트 하우스'와 '료칸'과 '호텔'이라는 연관 검색어를 보여줄 수 없습니다.

***이러한 단점을 없앨 수 있는, 단어의 '의미'를 다차원 공간에 벡터화 하는 기법으로는 두 가지***가 있습니다.

- 첫째는 카운트 기반으로 단어의 의미를 벡터화하는 LSA, HAL 등이 있으며,
- 둘째는 예측 기반으로 단어의 의미를 벡터화하는 전통 NNLM, RNNLM, Word2Vec, FastText 등이 있습니다. 
- 그리고 카운트 기반과 예측 기반 두 가지 방법을 모두 사용하는 방법으로 Glove라는 방법이 존재합니다.

이 책에서는 이 중에서 6챕터에서 LSA를 다룰 예정이며, 9챕터에서는 Word2Vec, FastText, Glove를 다룹니다.

# **단어 분리(Subword Segmentation)**
언어 모델(Language Model), 기계 번역(Neural Machine Translation) 등. 결국 머신 러닝을 이용한 자연어 처리의 목표는 기계가 사람 이상의 성능을 내는 것을 기대하는 것입니다. 그런데 아무리 많은 단어들을 기계에게 학습시켜도, 기계에게 세상의 모든 단어를 다 알려줄 수는 없는 노릇입니다. 그리고 사실 기계에게 더 많은 단어를 알려주려고 하면, 그만큼 계산 비용도 늘어난다는 부담이 있습니다.
<br>

기계가 훈련 단계에서 학습한 단어들의 집합을 단어 집합(vocabulary)이라고 합니다. 기계가 암기한 단어들의 리스트라고 생각할 수도 있겠습니다. 그리고 테스트 단계에서 기계가 미처 배우지 못한 모르는 단어가 등장한다면 이 단어들을 OOV(Out-Of-Vocabulary)라고 합니다. 단어 집합에 없는 단어라는 의미입니다. 이를 UNK(Unknown Word)로 표현하기도 합니다. 결국 기계가 모르는 단어로 인해 문제를 풀지 못하는 상황을 OOV 문제라고 합니다.
<br>

갑자기 용어들이 나와서 어려워보이지만 굉장히 단순한 이야기입니다. 사람에 비유하면 통번역사를 꿈꾸는 사람이 한국어 문장들과 그에 대응되는 영어 문장들을 나름 공부했더라도, 배운 적 없는 생소한 한국어 단어에 대해서 통역을 해보라고하면 못 하겠지요. 당장 한국어 단어의 의미도 모르겠는데 어떻게 번역하겠습니까? 여기서 그 단어는 그 사람에게 있어 OOV에 해당됩니다. 모르는 단어니까요.
<br>

내부 단어 분리(Subword Segmentation)는 기계가 아직 배운 적이 없는 단어더라도 대처할 수 있도록 도와주는 기법입니다. 이 방법은 이제는 기계 번역 등에서 주요 전처리로 사용되고 있습니다. 지금부터 단어 분리를 통해 OOV 문제를 해결하는 방법인 BPE(Byte Pair Encoding)과 WPM(Word Piece Model)에 대해서 학습해봅시다.
<br>

## **1. BPE(Byte Pair Encoding) 알고리즘**
BPE(Byte pair encoding) 알고리즘은 1994년에 제안된 데이터 압축 알고리즘입니다. 하지만 후에 자연어 처리의 단어 분리 알고리즘으로 응용되었는데, 어떻게 응용되었는지는 뒤에서 다시 언급하도록 하고, 우선 BPE 알고리즘의 작동 방법에 대해서 이해해보겠습니다. 아래와 같은 문자열이 주어졌을 때, BPE 알고리즘을 수행한다고 해봅시다.

    aaabdaaabac

BPE 알고리즘은 기본적으로 연속적으로 가장 많이 등장한 글자의 쌍을 찾아서 하나의 글자로 병합하는 방식을 수행합니다. 태생이 압축 알고리즘인 만큼, 여기서는 글자 대신 바이트(byte)라는 표현을 사용하겠습니다. 예를 들어 위의 문자열 중 가장 자주 등장하고 있는 바이트의 쌍(byte pair)은 'aa'입니다. 이 'aa'라는 바이트의 쌍을 하나의 바이트인 'Z'로 치환해보겠습니다.

    ZabdZabac
    Z=aa

이제 위 문자열 중에서 가장 많이 등장하고 있는 바이트의 쌍은 'ab'입니다. 이제 이 'ab'를 'Y'로 치환해봅시다.

    ZYdZYac
    Y=ab
    Z=aa

이제 가장 많이 등장하고 있는 바이트의 쌍은 'ZY'입니다. 이를 'X'로 치환해봅시다.

    XdXac
    X=ZY
    Y=ab
    Z=aa

이제 더 이상 병합할 바이트의 쌍은 없으므로 BPE 알고리즘은 종료됩니다.

## **2. BPE(Byte Pair Encoding) 알고리즘을 자연어 처리에 적용하기**
**Neural Machine Translation of Rare Words with Subword Units [Sennrich at el.2015]**라는 논문에서 자연어 처리. 그 중에서도 기계 번역을 위해 BPE 알고리즘을 사용할 것 제안하였습니다. 그리고 현재에 이르러 BPE 알고리즘은 자연어 처리를 위한 주요 전처리 방법으로 사용되고 있습니다. 앞으로 언급하는 BPE 알고리즘은 자연어 처리에서의 BPE 알고리즘입니다.
<br>

BPE 알고리즘은 단어 분리(word segmentation) 알고리즘입니다. 기존에 있던 단어를 분리한다는 의미인데, 어떤 의미인지는 뒤에서 나올 BPE 알고리즘의 최종 결과를 보면 이해할 수 있습니다. BPE 알고리즘을 요약하면, 글자(charcter) 단위에서 점차적으로 단어 집합(vocabulary)을 만들어 내는 Bottom up 방식의 접근을 사용합니다. 우선 훈련 데이터에 있는 단어들을 모든 글자(Chracters) 또는 유니코드(Unicode) 단위로 단어 집합(vocabulary)를 만들고, 가장 많이 등장하는 유니그램을 하나의 유니그램으로 통합합니다.
<br>

방금 제목을 언급한 논문에서 이미 BPE 알고리즘의 코드를 공개하였기 때문에, 따로 코드를 작성할 필요없이 손쉽게 파이썬 코드 실습이 가능합니다. 하지만 코드 실습을 진행하기 전에 육안으로 확인할 수 있는 간단한 예를 들어보겠습니다.

### **1) 기존의 접근**
어떤 훈련 데이터로부터 각 단어들의 빈도수를 카운트했다고 해보겠습니다. 그리고 각 단어와 각 단어의 빈도수가 기록되어져 있는 해당 결과는 임의로 딕셔너리(dictionary)란 이름을 붙였습니다.

* 딕셔너리 형태의 훈련 데이터에 있는 단어와 등장 빈도수 <br>
        low : 5, lower : 2, newest : 6, widest : 3

이 훈련 데이터에는 'low'란 단어가 5회 등장하였고, 'lower'란 단어는 2회 등장하였으며, 'newest'란 단어는 6회, 'widest'란 단어는 3회 등장하였다는 의미입니다. 그렇다면 딕셔너리로부터 이 훈련 데이터의 단어 집합(vocabulary)을 얻는 것은 간단합니다.

* vocabulary
        low, lower, newest, widest

단어 집합은 중복을 배제한 단어들의 집합을 의미하므로 기존에 배운 단어 집합의 정의라면, 이 훈련 데이터의 단어 집합에는 'low', 'lower', 'newest', 'widest'라는 4개의 단어가 존재합니다. 그리고 이 경우 테스트 과정에서 'lowest'란 단어가 등장한다면 기계는 이 단어를 학습한 적이 없으므로 해당 단어에 대해서 제대로 대응하지 못하는 OOV 문제가 발생합니다.

### **2) BPE 알고리즘을 적용**
BPE 알고리즘을 적용해서 얻은 단어 집합은 이러한 OOV에 대응할 수 있습니다. 이제 위의 딕셔너리에 BPE 알고리즘을 적용해봅시다. 우선 딕셔너리의 모든 단어들을 글자(chracter) 단위로 분리합니다. 이 경우 딕셔너리는 아래와 같습니다. 이제부터 딕셔너리는 자신 또한 업데이트되며 앞으로 단어 집합을 업데이트하기 위해 지속적으로 참고되는 참고 자료의 역할을 합니다.

* dictionary
        l o w : 5,  l o w e r : 2,  n e w e s t : 6,  w i d e s t : 3

딕셔너리를 참고로 한 초기 단어 집합(vocabulary)을 아래와 같습니다. 간단히 말해 초기 구성은 글자 단위로 분리된 상태입니다.

* vocabulary
        l, o, w, e, r, n, w, s, t, i, d

BPE 알고리즘의 특징은 알고리즘의 동작을 몇 회 반복(iteration)할 것인지를 사용자가 정해야 한다는 겁니다. 여기서는 총 10회를 수행한다고 가정합니다. 다시 말해 가장 빈도수가 높은 유니그램의 쌍을 하나의 유니그램으로 통합하는 과정을 총 10회 반복합니다. 위의 딕셔너리에 따르면 빈도수가 현재 가장 높은 유니그램의 쌍은 (e, s)입니다.

**1회 - 딕셔너리를 참고로 하였을 때 빈도수가 9로 가장 높은 (e, s)의 쌍을 es로 통합합니다.**
* dictionary update!
        l o w : 5,
        l o w e r : 2,
        n e w es t : 6,
        w i d es t : 3

* vocabulary update!
        l, o, w, e, r, n, w, s, t, i, d, es

**2회 - 빈도수가 9로 가장 높은 (es, t)의 쌍을 est로 통합합니다.**
* dictionary update!
        l o w : 5,
        l o w e r : 2,
        n e w est : 6,
        w i d est : 3

* vocabulary update!
        l, o, w, e, r, n, w, s, t, i, d, es, est

**3회 - 빈도수가 7로 가장 높은 (l, o)의 쌍을 lo로 통합합니다.**
* dictionary update!
        lo w : 5,
        lo w e r : 2,
        n e w est : 6,
        w i d est : 3
        
* vocabulary update!
        l, o, w, e, r, n, w, s, t, i, d, es, est, lo

이와 같은 방식으로 총 10회 반복하였을 때 얻은 딕셔너리와 단어 집합은 아래와 같습니다.

* dictionary update!
        low : 5,
        low e r : 2,
        newest : 6,
        widest : 3

* vocabulary update!
        l, o, w, e, r, n, w, s, t, i, d, es, est, lo, low, ne, new, newest, wi, wid, widest

이 경우 테스트 과정에서 'lowest'란 단어가 등장한다면, 기존에는 OOV에 해당되는 단어가 되었겠지만 BPE 알고리즘을 사용한 위의 단어 집합에서는 더 이상 'lowest'는 OOV가 아닙니다. 기계는 우선 'lowest'를 전부 글자 단위로 분할합니다. 즉, 'l, o, w, e, s, t'가 됩니다. 그리고 기계는 위의 단어 집합을 참고로 하여 'low'와 'est'를 찾아냅니다. 즉, 'lowest'를 기계는 'low'와 'est' 두 단어로 인코딩합니다. 그리고 이 두 단어는 둘 다 단어 집합에 있는 단어이므로 OOV가 아닙니다.

### **3) 코드 실습하기**

In [31]:
import re, collections

def get_stats(vocab):
    pairs = collections.defaultdict(int)
    for word, freq in vocab.items():
        symbols = word.split()
        for i in range(len(symbols) -1):
            pairs[symbols[i], symbols[i + 1]] += freq
    return pairs

def merge_vocab(pair, v_in):
    v_out = {}
    bigram = re.escape(' '.join(pair))
    p = re.compile(r'(?<!\S)' + bigram + r'(?!\S)')
    for word in v_in:
        w_out = p.sub(''.join(pair), word)
        v_out[w_out] = v_in[word]
    return v_out

vocab = {'l o w </w>' : 5,
         'l o w e r </w>' : 2,
         'n e w e s t </w>':6,
         'w i d e s t </w>':3
         }

num_merges = 10

for i in range(num_merges):
    pairs = get_stats(vocab)
    best = max(pairs, key = pairs.get)
    vocab = merge_vocab(best, vocab)
    print(best)

('e', 's')
('es', 't')
('est', '</w>')
('l', 'o')
('lo', 'w')
('n', 'e')
('ne', 'w')
('new', 'est</w>')
('low', '</w>')
('w', 'i')


BPE 알고리즘의 기본 원리는 가장 많이 등장한 문자열에 대하여 병합 하는 작업을 반복하는데, 원하는 단어 집합의 크기. 즉, 단어의 갯수가 될 때까지 이 작업을 반복합니다.

## **1. 단어 분리(Subword Segmentation)가 의미 있는 이유**

단어 분리(Subword segmenation) 작업은 하나의 단어는 의미있는 여러 단어들의 조합으로 구성된 경우가 많기 때문에, 단어를 여러 단어로 분리해보겠다는 전처리 작업입니다. 실제로, 언어의 특성에 따라 영어권 언어나 한국어는 단어 분리를 시도했을 때 어느정도 의미있는 단위로 나누는 것이 가능합니다.

## **2. WPM(Word Piece Model)**

WPM(Word Piece Model)은 하나의 단어를 내부 단어(Subword Unit)들로 분리하는 단어 분리 모델입니다.

실제로 2016년에 구글이 낸 논문**(Google’s Neural Machine Translation System: Bridging the Gap between Human and Machine Translation [Wo at el.2016])**에서 구글은 자신들의 구글 번역기에서 WPM이 어떻게 수행되는지에 대해서 기술하였습니다.

**WPM을 수행하기 이전의 문장:** Jet makers feud over seat width with big orders at stake
**WPM을 수행한 결과(wordpieces):** _J et _makers _fe ud _over _seat _width _with _big _orders _at _stake

Jet는 J와 et로 나누어졌으며, feud는 fe와 ud로 나누어진 것을 볼 수 있습니다. WPM은 입력 문장에서 기존에 존재하던 띄어쓰기는 언더바로 치환하고, 단어는 내부단어(subword)로 통계에 기반하여 띄어쓰기로 분리합니다. 기존의 띄어쓰기를 언더바로 치환하는 이유는 차후 다시 문장 복원을 위한 장치입니다. WPM의 결과로 나온 문장을 보면, 기존에 없던 띄어쓰기가 추가되어 내부 단어(subwords)들을 구분하는 구분자 역할을 하고 있으므로 본래의 띄어쓰기를 언더바로 치환해놓지 않으면, 기존 문장으로 복원할 수가 없습니다. WPM이 수행된 결과로부터 다시 수행 전의 결과로 돌리는 것 방법은 현재 있는 띄어쓰기를 전부 삭제하여 내부 단어들을 다시 하나의 단어로 연결시키고, 언더바를 다시 띄어쓰기로 바꾸면 됩니다.

------

논문 저자의 깃허브 : https://github.com/rsennrich/subword-nmt
참고 자료 : https://bab2min.tistory.com/622